# 07 cnn architectures keras
**Location: TensorVerseHub/notebooks/03_computer_vision/07_cnn_architectures_keras.ipynb**

In [ ]:
import tensorflow as tf
import numpy as np
print(f"TensorFlow version: {tf.__version__}")

# CNN Architectures with tf.keras

**File Location:** `notebooks/03_computer_vision/07_cnn_architectures_keras.ipynb`

Master convolutional neural networks with tf.keras, implementing classic and modern CNN architectures from LeNet to EfficientNet. Build custom CNN layers, understand architectural patterns, and create state-of-the-art computer vision models.

## Learning Objectives
- Implement classic CNN architectures (LeNet, AlexNet, VGG, ResNet)
- Build modern efficient architectures (MobileNet, EfficientNet concepts)
- Master tf.keras.layers.Conv2D and pooling operations
- Create custom convolutional layers and blocks
- Understand architectural design patterns and best practices
- Optimize CNNs for different computational constraints

---

## 1. CNN Fundamentals with tf.keras

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, applications
import os
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
tf.random.set_seed(42)
np.random.seed(42)

# Load and prepare datasets
def load_vision_datasets():
    """Load CIFAR-10 and Fashion-MNIST for CNN experiments"""
    
    # CIFAR-10 dataset
    (x_train_cifar, y_train_cifar), (x_test_cifar, y_test_cifar) = tf.keras.datasets.cifar10.load_data()
    x_train_cifar = x_train_cifar.astype('float32') / 255.0
    x_test_cifar = x_test_cifar.astype('float32') / 255.0
    y_train_cifar = y_train_cifar.flatten()
    y_test_cifar = y_test_cifar.flatten()
    
    # Fashion-MNIST dataset
    (x_train_fashion, y_train_fashion), (x_test_fashion, y_test_fashion) = tf.keras.datasets.fashion_mnist.load_data()
    x_train_fashion = x_train_fashion.astype('float32') / 255.0
    x_test_fashion = x_test_fashion.astype('float32') / 255.0
    # Add channel dimension
    x_train_fashion = np.expand_dims(x_train_fashion, axis=-1)
    x_test_fashion = np.expand_dims(x_test_fashion, axis=-1)
    
    print(f"CIFAR-10: Train {x_train_cifar.shape}, Test {x_test_cifar.shape}")
    print(f"Fashion-MNIST: Train {x_train_fashion.shape}, Test {x_test_fashion.shape}")
    
    return (x_train_cifar, y_train_cifar, x_test_cifar, y_test_cifar), \
           (x_train_fashion, y_train_fashion, x_test_fashion, y_test_fashion)

# Load datasets
cifar_data, fashion_data = load_vision_datasets()
(x_train_cifar, y_train_cifar, x_test_cifar, y_test_cifar) = cifar_data
(x_train_fashion, y_train_fashion, x_test_fashion, y_test_fashion) = fashion_data

# Basic CNN operations demonstration
def demonstrate_cnn_operations():
    """Demonstrate fundamental CNN operations"""
    
    print("=== CNN Operations Demonstration ===")
    
    # Create sample input
    sample_input = tf.random.normal((1, 32, 32, 3))  # Batch size 1, 32x32x3 image
    
    # Convolution operation
    conv_layer = layers.Conv2D(64, (3, 3), activation='relu', padding='same')
    conv_output = conv_layer(sample_input)
    print(f"Conv2D output shape: {conv_output.shape}")
    
    # Pooling operation
    pool_layer = layers.MaxPool2D((2, 2))
    pool_output = pool_layer(conv_output)
    print(f"MaxPool2D output shape: {pool_output.shape}")
    
    # Batch normalization
    bn_layer = layers.BatchNormalization()
    bn_output = bn_layer(pool_output)
    print(f"BatchNorm output shape: {bn_output.shape}")
    
    # Different convolution types
    print("\n--- Different Convolution Types ---")
    
    # Depthwise separable convolution
    depthwise_conv = layers.SeparableConv2D(64, (3, 3), padding='same')
    depthwise_output = depthwise_conv(sample_input)
    print(f"SeparableConv2D output shape: {depthwise_output.shape}")
    
    # Dilated convolution
    dilated_conv = layers.Conv2D(64, (3, 3), dilation_rate=2, padding='same')
    dilated_output = dilated_conv(sample_input)
    print(f"Dilated Conv2D output shape: {dilated_output.shape}")
    
    # Transpose convolution (for upsampling)
    transpose_conv = layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same')
    transpose_output = transpose_conv(sample_input)
    print(f"Conv2DTranspose output shape: {transpose_output.shape}")

demonstrate_cnn_operations()

## 2. Classic CNN Architectures

In [ ]:
# LeNet-5 Architecture
class LeNet5(tf.keras.Model):
    """LeNet-5 architecture for digit recognition"""
    
    def __init__(self, num_classes=10, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        
        # Convolutional layers
        self.conv1 = layers.Conv2D(6, (5, 5), activation='tanh', padding='valid')
        self.pool1 = layers.AveragePooling2D((2, 2))
        self.conv2 = layers.Conv2D(16, (5, 5), activation='tanh', padding='valid')
        self.pool2 = layers.AveragePooling2D((2, 2))
        
        # Dense layers
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(120, activation='tanh')
        self.fc2 = layers.Dense(84, activation='tanh')
        self.fc3 = layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)

In [ ]:
# AlexNet-inspired Architecture
class AlexNet(tf.keras.Model):
    """AlexNet-inspired architecture adapted for CIFAR-10"""
    
    def __init__(self, num_classes=10, **kwargs):
        super().__init__(**kwargs)
        
        self.conv1 = layers.Conv2D(96, (3, 3), strides=1, activation='relu', padding='same')
        self.bn1 = layers.BatchNormalization()
        self.pool1 = layers.MaxPool2D((2, 2), strides=2)
        
        self.conv2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')
        self.bn2 = layers.BatchNormalization()
        self.pool2 = layers.MaxPool2D((2, 2), strides=2)
        
        self.conv3 = layers.Conv2D(384, (3, 3), activation='relu', padding='same')
        self.conv4 = layers.Conv2D(384, (3, 3), activation='relu', padding='same')
        self.conv5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')
        self.pool5 = layers.MaxPool2D((2, 2), strides=2)
        
        self.flatten = layers.Flatten()
        self.dropout1 = layers.Dropout(0.5)
        self.fc1 = layers.Dense(4096, activation='relu')
        self.dropout2 = layers.Dropout(0.5)
        self.fc2 = layers.Dense(4096, activation='relu')
        self.fc3 = layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs, training=None):
        x = self.pool1(self.bn1(self.conv1(inputs)))
        x = self.pool2(self.bn2(self.conv2(x)))
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool5(self.conv5(x))
        
        x = self.flatten(x)
        x = self.dropout1(x, training=training)
        x = self.fc1(x)
        x = self.dropout2(x, training=training)
        x = self.fc2(x)
        return self.fc3(x)

In [ ]:
# VGG Architecture
class VGGBlock(layers.Layer):
    """VGG convolutional block"""
    
    def __init__(self, filters, num_convs, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.num_convs = num_convs
        
        self.convs = []
        for _ in range(num_convs):
            self.convs.append(layers.Conv2D(filters, (3, 3), activation='relu', padding='same'))
        
        self.pool = layers.MaxPool2D((2, 2))
    
    def call(self, inputs):
        x = inputs
        for conv in self.convs:
            x = conv(x)
        return self.pool(x)

class VGG16(tf.keras.Model):
    """VGG-16 architecture"""
    
    def __init__(self, num_classes=10, **kwargs):
        super().__init__(**kwargs)
        
        # VGG blocks
        self.block1 = VGGBlock(64, 2)
        self.block2 = VGGBlock(128, 2)
        self.block3 = VGGBlock(256, 3)
        self.block4 = VGGBlock(512, 3)
        self.block5 = VGGBlock(512, 3)
        
        # Classifier
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(4096, activation='relu')
        self.dropout1 = layers.Dropout(0.5)
        self.fc2 = layers.Dense(4096, activation='relu')
        self.dropout2 = layers.Dropout(0.5)
        self.fc3 = layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs, training=None):
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        
        x = self.flatten(x)
        x = self.dropout1(self.fc1(x), training=training)
        x = self.dropout2(self.fc2(x), training=training)
        return self.fc3(x)

In [ ]:
# Test classic architectures
print("\n=== Testing Classic CNN Architectures ===")

# LeNet on Fashion-MNIST
lenet = LeNet5(num_classes=10)
lenet.build(input_shape=(None, 28, 28, 1))
print(f"LeNet-5 parameters: {lenet.count_params():,}")

# AlexNet on CIFAR-10
alexnet = AlexNet(num_classes=10)
alexnet.build(input_shape=(None, 32, 32, 3))
print(f"AlexNet parameters: {alexnet.count_params():,}")

# VGG-16 (smaller version for CIFAR-10)
vgg16 = VGG16(num_classes=10)
vgg16.build(input_shape=(None, 32, 32, 3))
print(f"VGG-16 parameters: {vgg16.count_params():,}")

# Quick training test on LeNet with Fashion-MNIST
lenet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Use subset for quick test
subset_size = 1000
lenet_history = lenet.fit(
    x_train_fashion[:subset_size], y_train_fashion[:subset_size],
    validation_data=(x_test_fashion[:200], y_test_fashion[:200]),
    epochs=5, batch_size=32, verbose=0
)

print(f"LeNet-5 test accuracy: {max(lenet_history.history['val_accuracy']):.4f}")

## 3. Residual Networks (ResNet)

In [ ]:
# ResNet Implementation
class ResidualBlock(layers.Layer):
    """Basic residual block with skip connection"""
    
    def __init__(self, filters, strides=1, use_projection=False, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.strides = strides
        self.use_projection = use_projection
        
        # Main path
        self.conv1 = layers.Conv2D(filters, (3, 3), strides=strides, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(filters, (3, 3), padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # Skip connection projection if needed
        if use_projection:
            self.projection = layers.Conv2D(filters, (1, 1), strides=strides, padding='same')
            self.proj_bn = layers.BatchNormalization()
        
        self.relu = layers.ReLU()
        self.add = layers.Add()
    
    def call(self, inputs, training=None):
        # Main path
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        
        # Skip connection
        shortcut = inputs
        if self.use_projection:
            shortcut = self.projection(shortcut)
            shortcut = self.proj_bn(shortcut, training=training)
        
        # Add and activate
        x = self.add([x, shortcut])
        return self.relu(x)

class BottleneckBlock(layers.Layer):
    """Bottleneck residual block for deeper networks"""
    
    def __init__(self, filters, strides=1, use_projection=False, **kwargs):
        super().__init__(**kwargs)
        
        # Bottleneck: 1x1 -> 3x3 -> 1x1
        self.conv1 = layers.Conv2D(filters // 4, (1, 1), padding='same')
        self.bn1 = layers.BatchNormalization()
        
        self.conv2 = layers.Conv2D(filters // 4, (3, 3), strides=strides, padding='same')
        self.bn2 = layers.BatchNormalization()
        
        self.conv3 = layers.Conv2D(filters, (1, 1), padding='same')
        self.bn3 = layers.BatchNormalization()
        
        # Projection for skip connection
        if use_projection:
            self.projection = layers.Conv2D(filters, (1, 1), strides=strides, padding='same')
            self.proj_bn = layers.BatchNormalization()
        
        self.use_projection = use_projection
        self.relu = layers.ReLU()
        self.add = layers.Add()
    
    def call(self, inputs, training=None):
        # Main path
        x = self.relu(self.bn1(self.conv1(inputs), training=training))
        x = self.relu(self.bn2(self.conv2(x), training=training))
        x = self.bn3(self.conv3(x), training=training)
        
        # Skip connection
        shortcut = inputs
        if self.use_projection:
            shortcut = self.proj_bn(self.projection(shortcut), training=training)
        
        # Add and activate
        x = self.add([x, shortcut])
        return self.relu(x)

class ResNet(tf.keras.Model):
    """ResNet architecture"""
    
    def __init__(self, layers_config, num_classes=10, use_bottleneck=False, **kwargs):
        super().__init__(**kwargs)
        self.layers_config = layers_config
        self.num_classes = num_classes
        self.use_bottleneck = use_bottleneck
        
        # Initial convolution
        self.conv1 = layers.Conv2D(64, (7, 7), strides=2, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.pool1 = layers.MaxPool2D((3, 3), strides=2, padding='same')
        
        # Residual layers
        self.res_layers = []
        filters = 64
        
        for i, num_blocks in enumerate(layers_config):
            layer_blocks = []
            strides = 1 if i == 0 else 2
            
            # First block may need projection
            block_class = BottleneckBlock if use_bottleneck else ResidualBlock
            layer_blocks.append(block_class(filters, strides, use_projection=True))
            
            # Remaining blocks
            for _ in range(1, num_blocks):
                layer_blocks.append(block_class(filters))
            
            self.res_layers.append(layer_blocks)
            filters *= 2
        
        # Final layers
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs, training=None):
        # Initial processing
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)
        x = self.pool1(x)
        
        # Residual layers
        for layer_blocks in self.res_layers:
            for block in layer_blocks:
                x = block(x, training=training)
        
        # Classification
        x = self.global_pool(x)
        return self.classifier(x)

# Create different ResNet variants
def create_resnet_variants():
    """Create different ResNet architectures"""
    
    variants = {
        'ResNet-18': ResNet([2, 2, 2, 2], num_classes=10, use_bottleneck=False),
        'ResNet-34': ResNet([3, 4, 6, 3], num_classes=10, use_bottleneck=False),
        'ResNet-50': ResNet([3, 4, 6, 3], num_classes=10, use_bottleneck=True),
        'ResNet-101': ResNet([3, 4, 23, 3], num_classes=10, use_bottleneck=True)
    }
    
    print("=== ResNet Variants ===")
    
    for name, model in variants.items():
        model.build(input_shape=(None, 32, 32, 3))  # CIFAR-10 input shape
        print(f"{name}: {model.count_params():,} parameters")
    
    return variants

resnet_variants = create_resnet_variants()

# Train ResNet-18 on subset of CIFAR-10
resnet18 = resnet_variants['ResNet-18']
resnet18.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

subset_size = 2000
resnet_history = resnet18.fit(
    x_train_cifar[:subset_size], y_train_cifar[:subset_size],
    validation_data=(x_test_cifar[:400], y_test_cifar[:400]),
    epochs=10, batch_size=64, verbose=0
)

print(f"ResNet-18 test accuracy: {max(resnet_history.history['val_accuracy']):.4f}")

## 4. Modern Efficient Architectures

In [ ]:
# Depthwise Separable Convolution Block
class DepthwiseSeparableConv(layers.Layer):
    """Depthwise separable convolution block"""
    
    def __init__(self, filters, strides=1, **kwargs):
        super().__init__(**kwargs)
        
        self.depthwise = layers.DepthwiseConv2D((3, 3), strides=strides, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()
        
        self.pointwise = layers.Conv2D(filters, (1, 1), padding='same')
        self.bn2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU()
    
    def call(self, inputs, training=None):
        x = self.depthwise(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        
        x = self.pointwise(x)
        x = self.bn2(x, training=training)
        return self.relu2(x)

# MobileNet-inspired Architecture
class MobileNetV1(tf.keras.Model):
    """MobileNet V1 architecture"""
    
    def __init__(self, num_classes=10, width_multiplier=1.0, **kwargs):
        super().__init__(**kwargs)
        self.width_multiplier = width_multiplier
        
        # Standard convolution
        base_filters = int(32 * width_multiplier)
        self.conv1 = layers.Conv2D(base_filters, (3, 3), strides=2, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()
        
        # Depthwise separable convolutions
        self.ds_convs = []
        filter_sizes = [64, 128, 128, 256, 256, 512, 512, 512, 512, 512, 512, 1024, 1024]
        strides = [1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1]
        
        for filters, stride in zip(filter_sizes, strides):
            self.ds_convs.append(DepthwiseSeparableConv(
                int(filters * width_multiplier), strides=stride
            ))
        
        # Classifier
        self.global_pool = layers.GlobalAveragePooling2D()
        self.dropout = layers.Dropout(0.2)
        self.classifier = layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        
        for ds_conv in self.ds_convs:
            x = ds_conv(x, training=training)
        
        x = self.global_pool(x)
        x = self.dropout(x, training=training)
        return self.classifier(x)

# Squeeze-and-Excitation Block
class SEBlock(layers.Layer):
    """Squeeze-and-Excitation block"""
    
    def __init__(self, filters, se_ratio=16, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.se_ratio = se_ratio
        
        # Squeeze
        self.global_pool = layers.GlobalAveragePooling2D()
        
        # Excitation
        self.fc1 = layers.Dense(filters // se_ratio, activation='relu')
        self.fc2 = layers.Dense(filters, activation='sigmoid')
        
        # Reshape for broadcasting
        self.reshape = layers.Reshape((1, 1, filters))
        self.multiply = layers.Multiply()
    
    def call(self, inputs):
        # Squeeze: Global average pooling
        squeeze = self.global_pool(inputs)
        
        # Excitation: Two FC layers
        excitation = self.fc1(squeeze)
        excitation = self.fc2(excitation)
        excitation = self.reshape(excitation)
        
        # Scale: Element-wise multiplication
        return self.multiply([inputs, excitation])

# Inverted Residual Block (MobileNet V2)
class InvertedResidualBlock(layers.Layer):
    """Inverted residual block with linear bottleneck"""
    
    def __init__(self, filters, strides=1, expand_ratio=6, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.strides = strides
        self.expand_ratio = expand_ratio
        
        # Expansion phase
        expanded_filters = int(filters * expand_ratio)
        self.expand = layers.Conv2D(expanded_filters, (1, 1), padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU6()
        
        # Depthwise
        self.depthwise = layers.DepthwiseConv2D((3, 3), strides=strides, padding='same')
        self.bn2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU6()
        
        # Projection (linear)
        self.project = layers.Conv2D(filters, (1, 1), padding='same')
        self.bn3 = layers.BatchNormalization()
        
        # Skip connection
        self.use_skip = (strides == 1)
        self.add = layers.Add()
    
    def call(self, inputs, training=None):
        # Expansion
        x = self.expand(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        
        # Depthwise
        x = self.depthwise(x)
        x = self.bn2(x, training=training)
        x = self.relu2(x)
        
        # Projection
        x = self.project(x)
        x = self.bn3(x, training=training)
        
        # Skip connection
        if self.use_skip and x.shape == inputs.shape:
            x = self.add([x, inputs])
        
        return x

# EfficientNet-inspired Block
class MBConvBlock(layers.Layer):
    """Mobile Inverted Bottleneck Convolution block"""
    
    def __init__(self, filters, strides=1, expand_ratio=1, se_ratio=4, **kwargs):
        super().__init__(**kwargs)
        
        expanded_filters = int(filters * expand_ratio)
        
        # Expansion
        if expand_ratio != 1:
            self.expand = layers.Conv2D(expanded_filters, (1, 1), padding='same')
            self.bn1 = layers.BatchNormalization()
            self.activation1 = layers.Activation(tf.keras.utils.get_custom_objects().get('swish', 'relu'))
        else:
            self.expand = None
        
        # Depthwise
        self.depthwise = layers.DepthwiseConv2D((3, 3), strides=strides, padding='same')
        self.bn2 = layers.BatchNormalization()
        self.activation2 = layers.Activation(tf.keras.utils.get_custom_objects().get('swish', 'relu'))
        
        # Squeeze and Excitation
        if se_ratio > 0:
            self.se = SEBlock(expanded_filters, se_ratio)
        else:
            self.se = None
        
        # Output projection
        self.project = layers.Conv2D(filters, (1, 1), padding='same')
        self.bn3 = layers.BatchNormalization()
        
        # Skip connection
        self.use_skip = (strides == 1)
        self.dropout = layers.Dropout(0.2)
        self.add = layers.Add()
    
    def call(self, inputs, training=None):
        x = inputs
        
        # Expansion
        if self.expand is not None:
            x = self.expand(x)
            x = self.bn1(x, training=training)
            x = self.activation1(x)
        
        # Depthwise
        x = self.depthwise(x)
        x = self.bn2(x, training=training)
        x = self.activation2(x)
        
        # Squeeze and Excitation
        if self.se is not None:
            x = self.se(x)
        
        # Output projection
        x = self.project(x)
        x = self.bn3(x, training=training)
        
        # Skip connection with dropout
        if self.use_skip and x.shape == inputs.shape:
            x = self.dropout(x, training=training)
            x = self.add([x, inputs])
        
        return x

# Test modern architectures
print("\n=== Testing Modern Efficient Architectures ===")

# MobileNet V1
mobilenet = MobileNetV1(num_classes=10, width_multiplier=0.5)
mobilenet.build(input_shape=(None, 32, 32, 3))
print(f"MobileNet V1 (0.5x): {mobilenet.count_params():,} parameters")

# MobileNet with different width multipliers
width_multipliers = [0.25, 0.5, 0.75, 1.0]
print("\nMobileNet V1 Parameter Count by Width Multiplier:")
for wm in width_multipliers:
    mobile = MobileNetV1(num_classes=10, width_multiplier=wm)
    mobile.build(input_shape=(None, 32, 32, 3))
    print(f"  Width {wm}: {mobile.count_params():,} parameters")

# Train MobileNet on subset
mobilenet.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

mobile_history = mobilenet.fit(
    x_train_cifar[:1000], y_train_cifar[:1000],
    validation_data=(x_test_cifar[:200], y_test_cifar[:200]),
    epochs=8, batch_size=32, verbose=0
)

print(f"MobileNet V1 test accuracy: {max(mobile_history.history['val_accuracy']):.4f}")

## 5. Custom CNN Components and Advanced Techniques

In [ ]:
# Advanced CNN components
class MultiScaleConv(layers.Layer):
    """Multi-scale convolution with different kernel sizes"""
    
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        
        # Different kernel sizes
        self.conv1x1 = layers.Conv2D(filters // 4, (1, 1), padding='same')
        self.conv3x3 = layers.Conv2D(filters // 4, (3, 3), padding='same')
        self.conv5x5 = layers.Conv2D(filters // 4, (5, 5), padding='same')
        
        # Max pooling branch
        self.maxpool = layers.MaxPool2D((3, 3), strides=1, padding='same')
        self.pool_conv = layers.Conv2D(filters // 4, (1, 1), padding='same')
        
        # Batch normalization and activation
        self.bn = layers.BatchNormalization()
        self.activation = layers.ReLU()
        self.concat = layers.Concatenate()
    
    def call(self, inputs, training=None):
        # Parallel branches
        branch1 = self.conv1x1(inputs)
        branch2 = self.conv3x3(inputs)
        branch3 = self.conv5x5(inputs)
        
        branch4 = self.maxpool(inputs)
        branch4 = self.pool_conv(branch4)
        
        # Concatenate all branches
        x = self.concat([branch1, branch2, branch3, branch4])
        x = self.bn(x, training=training)
        return self.activation(x)

class DilatedConvBlock(layers.Layer):
    """Dilated convolution block for enlarged receptive field"""
    
    def __init__(self, filters, dilation_rates=[1, 2, 4], **kwargs):
        super().__init__(**kwargs)
        
        self.dilated_convs = []
        for rate in dilation_rates:
            self.dilated_convs.append(layers.Conv2D(
                filters // len(dilation_rates), (3, 3),
                dilation_rate=rate, padding='same'
            ))
        
        self.bn = layers.BatchNormalization()
        self.activation = layers.ReLU()
        self.concat = layers.Concatenate()
    
    def call(self, inputs, training=None):
        outputs = []
        for conv in self.dilated_convs:
            outputs.append(conv(inputs))
        
        x = self.concat(outputs)
        x = self.bn(x, training=training)
        return self.activation(x)

class AttentionModule(layers.Layer):
    """Channel and spatial attention module"""
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        # Channel attention
        self.channel_pool = layers.GlobalAveragePooling2D()
        self.channel_fc1 = layers.Dense(None)  # Will be set in build
        self.channel_fc2 = layers.Dense(None)  # Will be set in build
        
        # Spatial attention
        self.spatial_conv = layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid')
        
        self.multiply1 = layers.Multiply()
        self.multiply2 = layers.Multiply()
    
    def build(self, input_shape):
        channels = input_shape[-1]
        self.channel_fc1 = layers.Dense(channels // 8, activation='relu')
        self.channel_fc2 = layers.Dense(channels, activation='sigmoid')
        super().build(input_shape)
    
    def call(self, inputs):
        # Channel attention
        channel_avg = self.channel_pool(inputs)
        channel_att = self.channel_fc1(channel_avg)
        channel_att = self.channel_fc2(channel_att)
        channel_att = tf.expand_dims(tf.expand_dims(channel_att, 1), 1)
        
        x = self.multiply1([inputs, channel_att])
        
        # Spatial attention
        spatial_avg = tf.reduce_mean(x, axis=-1, keepdims=True)
        spatial_max = tf.reduce_max(x, axis=-1, keepdims=True)
        spatial_concat = tf.concat([spatial_avg, spatial_max], axis=-1)
        spatial_att = self.spatial_conv(spatial_concat)
        
        output = self.multiply2([x, spatial_att])
        return output

# Progressive Resizing Training
class ProgressiveResizeCallback(tf.keras.callbacks.Callback):
    """Progressive resizing during training"""
    
    def __init__(self, initial_size=32, target_size=224, resize_epochs=[10, 20]):
        super().__init__()
        self.sizes = [initial_size] + [target_size // (2 ** (len(resize_epochs) - i)) 
                                      for i in range(len(resize_epochs))] + [target_size]
        self.resize_epochs = [0] + resize_epochs + [float('inf')]
        self.current_size = initial_size
        
    def on_epoch_begin(self, epoch, logs=None):
        for i, resize_epoch in enumerate(self.resize_epochs[1:], 1):
            if epoch == resize_epoch and i < len(self.sizes):
                self.current_size = self.sizes[i]
                print(f"\nProgressive resize: switching to {self.current_size}x{self.current_size}")
                break

# Custom CNN with advanced components
class AdvancedCNN(tf.keras.Model):
    """CNN with advanced custom components"""
    
    def __init__(self, num_classes=10, **kwargs):
        super().__init__(**kwargs)
        
        # Initial convolution
        self.conv1 = layers.Conv2D(64, (7, 7), strides=2, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()
        self.pool1 = layers.MaxPool2D((3, 3), strides=2, padding='same')
        
        # Advanced blocks
        self.multiscale1 = MultiScaleConv(128)
        self.attention1 = AttentionModule()
        
        self.dilated_block = DilatedConvBlock(256, [1, 2, 4, 8])
        
        self.multiscale2 = MultiScaleConv(512)
        self.attention2 = AttentionModule()
        
        # Global pooling and classification
        self.global_pool = layers.GlobalAveragePooling2D()
        self.dropout = layers.Dropout(0.5)
        self.classifier = layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs, training=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.multiscale1(x, training=training)
        x = self.attention1(x)
        
        x = self.dilated_block(x, training=training)
        
        x = self.multiscale2(x, training=training)
        x = self.attention2(x)
        
        x = self.global_pool(x)
        x = self.dropout(x, training=training)
        return self.classifier(x)

# Test advanced CNN
print("\n=== Testing Advanced CNN Components ===")

advanced_cnn = AdvancedCNN(num_classes=10)
advanced_cnn.build(input_shape=(None, 32, 32, 3))
print(f"Advanced CNN parameters: {advanced_cnn.count_params():,}")

# Train with advanced techniques
advanced_cnn.compile(
    optimizer=tf.keras.optimizers.AdamW(0.001, weight_decay=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Use data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Train on subset with augmentation
advanced_history = advanced_cnn.fit(
    datagen.flow(x_train_cifar[:1000], y_train_cifar[:1000], batch_size=32),
    steps_per_epoch=30,
    epochs=10,
    validation_data=(x_test_cifar[:200], y_test_cifar[:200]),
    verbose=0
)

print(f"Advanced CNN test accuracy: {max(advanced_history.history['val_accuracy']):.4f}")

## 6. Architecture Comparison and Best Practices

In [ ]:
# Comprehensive architecture comparison
class CNNBenchmark:
    """Benchmark different CNN architectures"""
    
    def __init__(self):
        self.results = {}
        
    def benchmark_architecture(self, name, model, train_data, val_data, epochs=5):
        """Benchmark a single architecture"""
        
        print(f"\nBenchmarking {name}...")
        
        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Training
        import time
        start_time = time.time()
        
        history = model.fit(
            train_data[0], train_data[1],
            validation_data=val_data,
            epochs=epochs,
            batch_size=32,
            verbose=0
        )
        
        training_time = time.time() - start_time
        
        # Inference timing
        inference_times = []
        for _ in range(10):
            start = time.time()
            _ = model.predict(val_data[0][:100], verbose=0)
            inference_times.append(time.time() - start)
        
        avg_inference_time = np.mean(inference_times)
        
        # Store results
        self.results[name] = {
            'parameters': model.count_params(),
            'training_time': training_time,
            'inference_time': avg_inference_time,
            'best_val_accuracy': max(history.history['val_accuracy']),
            'final_val_accuracy': history.history['val_accuracy'][-1],
            'convergence_epoch': np.argmax(history.history['val_accuracy']) + 1
        }
        
        print(f"  Parameters: {self.results[name]['parameters']:,}")
        print(f"  Best Val Acc: {self.results[name]['best_val_accuracy']:.4f}")
        print(f"  Training Time: {training_time:.2f}s")
        
    def print_comparison(self):
        """Print detailed comparison table"""
        
        print("\n=== CNN Architecture Comparison ===")
        print(f"{'Architecture':<15} {'Params':<10} {'Best Acc':<10} {'Train Time':<12} {'Inference':<12} {'Convergence':<12}")
        print("-" * 80)
        
        for name, result in self.results.items():
            print(f"{name:<15} {result['parameters']:<10,} "
                  f"{result['best_val_accuracy']:<10.4f} "
                  f"{result['training_time']:<12.2f} "
                  f"{result['inference_time']:<12.4f} "
                  f"{result['convergence_epoch']:<12}")
        
        # Analysis
        print("\n=== Analysis ===")
        
        # Best accuracy
        best_acc_model = max(self.results.items(), key=lambda x: x[1]['best_val_accuracy'])
        print(f"Best Accuracy: {best_acc_model[0]} ({best_acc_model[1]['best_val_accuracy']:.4f})")
        
        # Most efficient (params)
        most_efficient = min(self.results.items(), key=lambda x: x[1]['parameters'])
        print(f"Most Efficient: {most_efficient[0]} ({most_efficient[1]['parameters']:,} params)")
        
        # Fastest training
        fastest_train = min(self.results.items(), key=lambda x: x[1]['training_time'])
        print(f"Fastest Training: {fastest_train[0]} ({fastest_train[1]['training_time']:.2f}s)")
        
        # Fastest inference
        fastest_inference = min(self.results.items(), key=lambda x: x[1]['inference_time'])
        print(f"Fastest Inference: {fastest_inference[0]} ({fastest_inference[1]['inference_time']:.4f}s)")

# Run comprehensive benchmark
benchmark = CNNBenchmark()

# Prepare smaller datasets for quick benchmarking
train_subset = (x_train_cifar[:800], y_train_cifar[:800])
val_subset = (x_test_cifar[:200], y_test_cifar[:200])

# Benchmark different architectures
architectures_to_test = {
    'LeNet-5': LeNet5(num_classes=10),
    'AlexNet': AlexNet(num_classes=10),
    'ResNet-18': resnet_variants['ResNet-18'],
    'MobileNet': MobileNetV1(num_classes=10, width_multiplier=0.5),
    'Advanced_CNN': AdvancedCNN(num_classes=10)
}

# Build all models first
for name, model in architectures_to_test.items():
    if name == 'LeNet-5':
        # LeNet expects grayscale, convert CIFAR-10 to grayscale
        gray_train = tf.image.rgb_to_grayscale(train_subset[0])
        gray_val = tf.image.rgb_to_grayscale(val_subset[0])
        gray_train = tf.image.resize(gray_train, [28, 28])
        gray_val = tf.image.resize(gray_val, [28, 28])
        model.build(input_shape=(None, 28, 28, 1))
        benchmark.benchmark_architecture(
            name, model, (gray_train, train_subset[1]), (gray_val, val_subset[1]), epochs=3
        )
    else:
        model.build(input_shape=(None, 32, 32, 3))
        benchmark.benchmark_architecture(name, model, train_subset, val_subset, epochs=3)

benchmark.print_comparison()

# CNN Design Best Practices
def print_cnn_best_practices():
    """Print CNN design best practices and guidelines"""
    
    print("\n" + "="*60)
    print("CNN ARCHITECTURE DESIGN BEST PRACTICES")
    print("="*60)
    
    practices = {
        "🏗️ ARCHITECTURAL PATTERNS": [
            "Use residual connections for networks deeper than 20 layers",
            "Apply batch normalization after convolutions, before activation",
            "Use ReLU or its variants (LeakyReLU, ELU, Swish) for activation",
            "Implement progressive downsampling with pooling or strided convs",
            "Consider depthwise separable convs for mobile/edge deployment"
        ],
        
        "⚡ EFFICIENCY OPTIMIZATIONS": [
            "Use global average pooling instead of large fully connected layers",
            "Apply channel/spatial attention for better feature selection",
            "Implement early stopping and learning rate scheduling",
            "Use mixed precision training for faster training on modern GPUs",
            "Consider progressive resizing for faster convergence"
        ],
        
        "📊 DATA HANDLING": [
            "Normalize input data to [0,1] or [-1,1] range",
            "Apply data augmentation (rotation, flip, zoom, crop)",
            "Use proper train/validation/test splits",
            "Implement data pipeline optimization with tf.data",
            "Consider mixup or cutmix for improved generalization"
        ],
        
        "🎯 REGULARIZATION TECHNIQUES": [
            "Apply dropout (0.2-0.5) in dense layers, not conv layers",
            "Use L2 weight decay (1e-4 to 1e-2) in optimizer",
            "Implement label smoothing for better calibration",
            "Consider stochastic depth for very deep networks",
            "Apply spectral normalization for improved training stability"
        ],
        
        "🔧 HYPERPARAMETER GUIDELINES": [
            "Start with learning rate 1e-3, adjust based on validation loss",
            "Use batch size 32-128 depending on memory constraints",
            "Apply cosine annealing or reduce-on-plateau LR scheduling",
            "Set initial weights with He or Xavier initialization",
            "Monitor gradient norms to detect vanishing/exploding gradients"
        ],
        
        "📱 DEPLOYMENT CONSIDERATIONS": [
            "Use quantization for mobile/edge deployment",
            "Consider pruning for model compression",
            "Implement model distillation for smaller student models",
            "Use TensorFlow Lite for mobile optimization",
            "Profile inference time vs accuracy trade-offs"
        ]
    }
    
    for category, items in practices.items():
        print(f"\n{category}")
        for item in items:
            print(f"  • {item}")
    
    print(f"\n{'='*60}")
    print("ARCHITECTURE SELECTION GUIDE")
    print("="*60)
    
    selection_guide = {
        "High Accuracy (Research)": "ResNet-50/101, EfficientNet, Vision Transformers",
        "Mobile/Edge Deployment": "MobileNet, EfficientNet-B0, SqueezeNet",
        "Real-time Applications": "YOLO variants, MobileNet, optimized ResNet",
        "Limited Data": "Transfer learning with pre-trained models",
        "Custom Domain": "Start with ResNet-18/34, add domain-specific layers"
    }
    
    for use_case, recommendation in selection_guide.items():
        print(f"{use_case:<25}: {recommendation}")

print_cnn_best_practices()

## Summary

**File Location:** `notebooks/03_computer_vision/07_cnn_architectures_keras.ipynb`

This comprehensive notebook mastered CNN architectures with tf.keras:

### Classic Architectures Implemented:
1. **LeNet-5**: Pioneer architecture for digit recognition
2. **AlexNet**: First deep CNN with ReLU and dropout
3. **VGG-16**: Deep networks with small 3x3 filters
4. **ResNet**: Residual connections enabling very deep networks

### Modern Efficient Architectures:
- **MobileNet V1**: Depthwise separable convolutions for mobile
- **Inverted Residual Blocks**: Linear bottlenecks for efficiency
- **Squeeze-and-Excitation**: Channel attention mechanisms
- **EfficientNet-inspired**: Compound scaling strategies

### Advanced CNN Components:
- **Multi-scale Convolutions**: Parallel filters of different sizes
- **Dilated Convolutions**: Enlarged receptive fields without parameters
- **Attention Modules**: Channel and spatial attention mechanisms
- **Progressive Training**: Dynamic input size scheduling

### Key Technical Insights:
- **Residual connections** prevent vanishing gradients in deep networks
- **Depthwise separable convolutions** reduce parameters by 8-9x
- **Batch normalization** accelerates training and improves stability
- **Global average pooling** reduces parameters vs fully connected layers
- **Attention mechanisms** improve feature selection and model interpretability

### Performance Optimizations:
- **Parameter efficiency**: MobileNet achieves good accuracy with fewer params
- **Training speed**: Batch normalization and proper initialization crucial
- **Inference speed**: Depthwise convolutions and efficient blocks
- **Memory usage**: Global pooling and architectural choices matter

### Best Practices Established:
- Use residual connections for networks >20 layers
- Apply batch normalization after conv, before activation
- Implement proper data augmentation and regularization
- Consider deployment constraints when choosing architecture
- Profile accuracy vs efficiency trade-offs

### Next Steps:
- Apply transfer learning with pre-trained models (Notebook 08)
- Implement semantic segmentation architectures (Notebook 09)
- Explore vision transformers and hybrid architectures
- Deploy optimized models to mobile and edge devices

This foundation enables building and optimizing CNN architectures for any computer vision task!